# Dataset Cleanup

This notebook demonstrates how the original 24 data files are combined into the 4 train and test sets provided.


# Notebook Preparation

In [ ]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')

# Obtaining our Data

In [ ]:
# Load a month of data so we can see what kind of information we're working with
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_01.csv')
df.shape

In [ ]:
df

In [ ]:
# Check memory usage of this file
df.memory_usage().sum()
# These files are LARGE, and this is only one month. 
# Part of our cleaning process will be to store our data in a more memory efficient manner.

In [ ]:
# What types of data do we have? We can get a feel of what we may be able to reduce to a smaller integer
df.dtypes

In [ ]:
# Descriptive stats for our data
df.describe()

In [ ]:
# Check our missing data
df.isna().sum()

# Scrubbing/Cleaning our Data

## Loading Data for Merging

In [ ]:
# Information on passenger activity for airports and airlines
passengers = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2019.csv')
passengers

In [ ]:
# Manufacture year and passenger capacity for aircraft by unique aircraft tail number
aircraft = pd.read_csv("../input/2019-airline-delays-and-cancellations/raw_data/B43_AIRCRAFT_INVENTORY.csv",encoding='latin1')
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True)
aircraft

In [ ]:
# coordinates of airports
coords = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/AIRPORT_COORDINATES.csv')
coords.drop_duplicates(subset='ORIGIN_AIRPORT_ID', inplace=True)
coords

In [ ]:
# proper names of carriers for better EDA usage
names = pd.read_csv("../input/2019-airline-delays-and-cancellations/raw_data/CARRIER_DECODE.csv")
names.drop_duplicates(inplace=True)
names.drop_duplicates(subset=['OP_UNIQUE_CARRIER'], inplace=True)
names

In [ ]:
# Employee statistics for carriers
employees = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/P10_EMPLOYEES.csv')
employees = employees[['OP_UNIQUE_CARRIER', 'PASS_GEN_SVC_ADMIN', 'PASSENGER_HANDLING']]
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index()
employees

### Cleaning Weather Data

In [ ]:
# Weather report for top 90% of airport cities, in 2019
weather_report = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/airport_weather_2019.csv')
weather_report

In [ ]:
# Our list of cities and airports including the airport display name so that we can connect with our main df
cities = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/airports_list.csv')
cities

In [ ]:
# Connect our weather report with the city names
weather_merge = pd.merge(cities, weather_report, how='left', on='NAME')
weather_merge

In [ ]:
# Get just the important metrics from the weather report (date, precipitation, snow, temp, wind)
weather = weather_merge[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]

In [ ]:
# Drop any rows where no weather was recorded
weather.drop(weather.loc[weather['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)

In [ ]:
# Look for null values in temperature
weather.loc[weather['TMAX'].isna()]

In [ ]:
# Impute mean in nan rows for temp and wind
weather['TMAX'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather['AWND'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather.fillna(0, inplace=True)

In [ ]:
# Check no NaN remain
weather.isna().sum()

In [ ]:
# Cast data types to datetime so we can get the month and day of month to match up with main df
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month
weather['DAY_OF_MONTH'] = pd.DatetimeIndex(weather['DATE']).day
weather

# Cleaning Function

In [ ]:
def month_cleanup(monthly_data, aircraft, coords, names, weather, passengers, employees):
    
    '''Function which performs features engineering, data merges and cleanup using one month of On-Time data 
    from Bureau of Transportation Services
    Parameters:
    monthly_data: month of on-time data as downloaded from BTS
    aircraft: Aircraft inventory data from BTS
    coords: Airport coordinates data from BTS
    names: Carrier names based on carrier code from BTS
    weather: Daily weather reported at airports from National Center for Environmental Information
    passengers: Yearly passenger information for carriers and airports from BTS
    employees: Employee statistics for carriers from BTS
    '''
    
    # start the timer so we can track how long the cleaning function takes
    start = time.time()
    
    
    # drop rows with no departure time, tail number, or were cancelled
    print("Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.")
    monthly_data.drop(monthly_data.loc[monthly_data['DEP_TIME'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['CANCELLED']==1].index, axis=0, inplace=True)
    print(f'Elapsed Time: {time.time() - start}')
   
    
    # Create time blocks for departure for cleaner categories
    print("\nCreating Departure Time Blocks - DEP_BLK")
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='2100-2159') | (monthly_data['DEP_TIME_BLK']=='2200-2259') | (monthly_data['DEP_TIME_BLK']=='2300-2359'), 'DEP_BLOCK'] = 'LATE_NIGHT'
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='0001-0559'), 'DEP_BLOCK'] = 'EARLY_MORNING'      
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='0600-0659') | (monthly_data['DEP_TIME_BLK']=='0700-0759') | (monthly_data['DEP_TIME_BLK']=='0800-0859') | (monthly_data['DEP_TIME_BLK']=='0900-0959'), 'DEP_BLOCK'] = 'MORNING'
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='1000-1059') | (monthly_data['DEP_TIME_BLK']=='1100-1159') | (monthly_data['DEP_TIME_BLK']=='1200-1259'), 'DEP_BLOCK'] = 'MIDDAY'
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='1300-1359') | (monthly_data['DEP_TIME_BLK']=='1400-1459') | (monthly_data['DEP_TIME_BLK']=='1500-1559') | (monthly_data['DEP_TIME_BLK']=='1600-1659'), 'DEP_BLOCK'] = 'AFTERNOON'
    monthly_data.loc[(monthly_data['DEP_TIME_BLK']=='1700-1759') | (monthly_data['DEP_TIME_BLK']=='1800-1859') | (monthly_data['DEP_TIME_BLK']=='1900-1959') | (monthly_data['DEP_TIME_BLK']=='2000-2059') , 'DEP_BLOCK'] = 'EVENING'
    print(f'Elapsed Time: {time.time() - start}')
   
    
    # List flight segment number for daily flight segments by tracking tail number
    print("\nAdding Flight Number Sequence - SEGMENT_NUMBER")
    monthly_data["SEGMENT_NUMBER"] = monthly_data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)
    print(f'Elapsed Time: {time.time() - start}') 
    
   
    # Listing the concurrent flights at the airport in the time block 
    print("\nAdding Concurrent Flights - CONCURRENT_FLIGHTS")
    monthly_data['CONCURRENT_FLIGHTS'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_BLOCK'])['OP_UNIQUE_CARRIER'].transform("count")
    print(f'Elapsed Time: {time.time() - start}')
 
    
    # Getting seat counts for each aircraft
    print("\nApplying seat counts to flights - NUMBER_OF_SEATS")   
    # Merge aircraft info with main frame on tail number
    monthly_data = pd.merge(monthly_data, aircraft, how="left", on='TAIL_NUM')
    # Fill missing aircraft info with means
    monthly_data['NUMBER_OF_SEATS'].fillna((monthly_data['NUMBER_OF_SEATS'].mean()), inplace=True)
    # simplify data type of number of seats to reduce memory usage
    monthly_data['NUMBER_OF_SEATS'] = monthly_data['NUMBER_OF_SEATS'].astype('int16')
    print(f'Elapsed Time: {time.time() - start}')

    
    # Merge proper carrier name
    print("\nApplying Carrier Names - CARRIER_NAME")  
    monthly_data = pd.merge(monthly_data, names, how='left', on=['OP_UNIQUE_CARRIER'])
    print(f'Elapsed Time: {time.time() - start}')    
    
    
    # Add monthly flight statistics for carrier and airport
    print("\nAdding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH")
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    print(f'Elapsed Time: {time.time() - start}')
    
    
    #Add monthly passenger statistics for carrier and airport
    print("\nAdding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE")
    monthly_airport_passengers = pd.DataFrame(passengers.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['AVG_MONTHLY_PASS_AIRPORT'] = (monthly_data['REV_PAX_ENP_110']/12).astype('int64')
    monthly_airline_passengers = pd.DataFrame(passengers.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['AVG_MONTHLY_PASS_AIRLINE'] = (monthly_data['REV_PAX_ENP_110_y']/12).astype('int64')
    print(f'Elapsed Time: {time.time() - start}')  
    
    
    # Add employee stats
    print("\nAdding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS")
    monthly_data = pd.merge(monthly_data, employees, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['FLT_ATTENDANTS_PER_PASS'] = monthly_data['PASSENGER_HANDLING']/monthly_data['REV_PAX_ENP_110_y']
    monthly_data['GROUND_SERV_PER_PASS'] = monthly_data['PASS_GEN_SVC_ADMIN']/monthly_data['REV_PAX_ENP_110_y']
    print(f'Elapsed Time: {time.time() - start}')   
    
    
    # Calculate age of plane
    print("\nCalculate Fleet Age - PLANE_AGE")
    monthly_data['MANUFACTURE_YEAR'].fillna((monthly_data['MANUFACTURE_YEAR'].mean()), inplace=True)
    monthly_data['PLANE_AGE'] = 2019 - monthly_data['MANUFACTURE_YEAR']
    print(f'Elapsed Time: {time.time() - start}') 

    
    # Merge airport coordinates
    print("\nAdding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT")
    monthly_data = pd.merge(monthly_data, coords, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['LATITUDE'] = round(monthly_data['LATITUDE'], 3)
    monthly_data['LONGITUDE'] = round(monthly_data['LONGITUDE'], 3)
    print(f'Elapsed Time: {time.time() - start}')

    
    # Get previous airport for tail number
    print("\nAdding airports - PREVIOUS_AIRPORT")
    segment_temp = monthly_data[['DAY_OF_MONTH', 'TAIL_NUM', 'DISPLAY_AIRPORT_NAME', 'SEGMENT_NUMBER']]
    monthly_data = pd.merge_asof(monthly_data.sort_values('SEGMENT_NUMBER'), segment_temp.sort_values('SEGMENT_NUMBER'), on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], allow_exact_matches=False)
    monthly_data['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    monthly_data.rename(columns={"DISPLAY_AIRPORT_NAME_y": "PREVIOUS_AIRPORT", "DISPLAY_AIRPORT_NAME_x": "DEPARTING_AIRPORT"}, inplace=True)  
    print(f'Elapsed Time: {time.time() - start}')
    
    
    # Drop airports below the 10th percentile
    print("\nDropping bottom 10% of airports")
    monthly_data.drop(monthly_data.loc[monthly_data['AIRPORT_FLIGHTS_MONTH'] < 1100].index, axis=0, inplace=True)
    print(f'Elapsed Time: {time.time() - start}')
    
    
    # Merge weather data
    print("\nAdding daily weather data - PRCP, SNOW, SNWD, SMAX, TMIN, AWND")
    monthly_data = pd.merge(monthly_data, weather, how='inner', on=['ORIGIN_AIRPORT_ID', 'MONTH', 'DAY_OF_MONTH'])
    
    print(f'Elapsed Time: {time.time() - start}')
    
    
    # drop columns that we won't use
    print("\nClean up unneeded columns")
    monthly_data.drop(columns = ['ORIGIN',  'DEST',  
                   'CRS_DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 
                   'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME', 'DISTANCE',
                   'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
                  'ARR_DELAY_NEW', 'Unnamed: 32', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'ACTUAL_ELAPSED_TIME',
                  'DEST_AIRPORT_ID', 'DEST_CITY_NAME',  'OP_CARRIER_FL_NUM',  'OP_UNIQUE_CARRIER',
                       'AIRLINE_ID', 'DATE', 'DAY_OF_MONTH', 'TAIL_NUM','DEP_TIME',
                    'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',  'PASSENGER_HANDLING', 'REV_PAX_ENP_110_x', 'REV_PAX_ENP_110_y', 
                                 'PASS_GEN_SVC_ADMIN', 'MANUFACTURE_YEAR',
                                 ],
                    axis=1, inplace=True) #,    
    print(f'Elapsed Time: {time.time() - start}') 
    
    
    # specify data types of various fields to reduce memory usage
    print("\nCleaning up data types")
    monthly_data['MONTH'] = monthly_data['MONTH'].astype('object')
    monthly_data['DAY_OF_WEEK'] = monthly_data['DAY_OF_WEEK'].astype('object')
    monthly_data['DEP_DEL15'] = monthly_data['DEP_DEL15'].astype('int8')
    monthly_data['DISTANCE_GROUP'] = monthly_data['DISTANCE_GROUP'].astype('int8')
    monthly_data['DEP_BLOCK'] = monthly_data['DEP_BLOCK'].astype('object')
    monthly_data['SEGMENT_NUMBER'] = monthly_data['SEGMENT_NUMBER'].astype('int8')
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data['AIRLINE_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['PLANE_AGE'] = monthly_data['PLANE_AGE'].astype('int32')

    monthly_data.reset_index(inplace=True, drop=True)
    
    print(f'Elapsed Time: {time.time() - start}')
    
    print("\nFINISHED")
    return monthly_data

# Process Train/Test Sets

Process each monthly set one at a time, then concat these into a master set. 

Month01 is saved as a small train/test set, and the concat of all months is saved as the large train/test set.

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_01.csv')
month01 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
month01.to_csv('./train_test_small.csv', index=False)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_02.csv')
month02 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_03.csv')
month03 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_04.csv')
month04 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_05.csv')
month05 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_06.csv')
month06 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_07.csv')
month07 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_08.csv')
month08 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_09.csv')
month09 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_10.csv')
month10 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_11.csv')
month11 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_12.csv')
month12 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)

In [ ]:
# COMBINE MASTER FILE

combined = pd.concat([month01, month02, month03, month04, month05, month06, month07, month08, month09, month10, month11, month12])
combined.to_csv('./train_test.csv', index=False)

# Process "New Data" Set

The "New Data" set is overall processed in the same way, except we need to use the weather data from these months. We make a small set of new data for just month01 of 2020, and a larger new data set for Jan-Mar

In [ ]:
# process weather from 2020 in same manner as 2019 weather
weather_report2 = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/airport_weather_2020.csv')
weather_merge2 = pd.merge(cities, weather_report2, how='left', on='NAME')
weather2 = weather_merge2[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'ORIGIN_AIRPORT_ID']]
weather2.drop(weather2.loc[weather2['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)
weather2['TMAX'].fillna(round(weather2.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather2['AWND'].fillna(round(weather2.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather2.fillna(0, inplace=True)
weather2['DATE'] = pd.to_datetime(weather2['DATE'])
weather2['MONTH'] = pd.DatetimeIndex(weather2['DATE']).month
weather2['DAY_OF_MONTH'] = pd.DatetimeIndex(weather2['DATE']).day
weather2

In [ ]:
# Import 2020 passenger data for airlines and airports
passengers2 = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2020.csv')
passengers2

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_2020_01.csv')
test01 = month_cleanup(df, aircraft, coords, names, weather2, passengers2, employees)
test01.to_csv("./new_data_small.csv", index=False)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_2020_02.csv')
test02 = month_cleanup(df, aircraft, coords, names, weather2, passengers2, employees)

In [ ]:
df = pd.read_csv('../input/2019-airline-delays-and-cancellations/raw_data/ONTIME_REPORTING_2020_03.csv')
test03 = month_cleanup(df, aircraft, coords, names, weather2, passengers2, employees)

In [ ]:
# COMBINE FILE
test = pd.concat([test01, test02, test03])
test.to_csv('./new_data.csv', index=False)